In [ ]:
# 거래량 급증

In [19]:
import requests
from bs4 import BeautifulSoup as bs

kospi_url = "https://finance.naver.com/sise/sise_quant_high.nhn?sosok=0"
kosdak_url = "https://finance.naver.com/sise/sise_quant_high.nhn?sosok=1"
list_url = [
    kospi_url, kosdak_url
]

def remake_list(list_base):
    num_cols = [3,4,6,7,8,9]
    rate_cols = [0,5,10]
    for idx in range(len(num_cols)):
        try:
            num = int(list_base[num_cols[idx]].replace(",",""))
        except:
            num = 0
        list_base[num_cols[idx]] = num
    for idx in range(len(rate_cols)):
        try:
            num = float(list_base[rate_cols[idx]].replace(",","").replace("%",""))
        except:
            num = 0
        list_base[rate_cols[idx]] = num
        
    return list_base


list_cols = [
    "증가율","종목코드","종목명","현재가","전일비","등락률","매수호가","매도호가","거래량","전일거래량","PER"
]        
list_whole = []

def get_data(base_url):
    response = requests.get( base_url )
    soup = bs(response.text, 'html.parser')

    content = soup.select("div.box_type_l")
    list_content = str(content).split("\n")
    list_jongmok = []
    for row_data in list_content:
        row = row_data.strip().replace("\t","")
        
        if '<td class="no">' in row:
            if len(list_jongmok) > 0:
                list_whole.append(list_jongmok)
                list_jongmok = []
        elif "tltle" in row:
            jongmok_cd = row.split("code=")[1].split('"')[0]
            jongmok_nm = row.split("code=")[1].split('"')[1].split('</a')[0].replace(">","")
            list_jongmok.append("A" + jongmok_cd)
            list_jongmok.append(jongmok_nm)
        elif row[:1] != "<":
            list_jongmok.append(row)
        elif ("<td" not in row or row[:4] == "<tr>"):
            continue
        else:
            num_val = row.replace('<td class="number">','')
            if len(num_val) == 0:
                continue
            list_jongmok.append(num_val.replace("</td>",""))


for url in list_url:
    get_data(url)
    
list_result = []
for list_row in list_whole:
    if len(list_row) == 11:
        list_result.append(remake_list(list_row))
        
head = """
insert into naver_vol_rising
(rising_vs_rt, jongmok_cd, jongmok_nm, prc, pre_vs_gap, pre_vs_rt, buy_prc, sell_prc, vol, pre_vol, per)
values
"""
body = ""
for list_row in list_result:
    if list_row[10] <= 0.0: continue
    row = "("
    for idx in range(len(list_row)):
        try:
            row += "'" + list_row[idx] + "', "
        except:
            row += str(list_row[idx]) + ", "
    row += ")," + "\n"
    row = row.replace(", )", ")")
    body += row

ins_qry = head + body
print(ins_qry[:len(ins_qry)-2])


insert into naver_vol_rising
(rising_vs_rt, jongmok_cd, jongmok_nm, prc, pre_vs_gap, pre_vs_rt, buy_prc, sell_prc, vol, pre_vol, per)
values
(10749.07, 'A007110', '일신석재', 2700, 395, 17.14, 2700, 2705, 114822689, 1058364, 180.0),
(5617.11, 'A138490', '코오롱플라스틱', 7760, 970, 14.29, 7760, 7770, 13120413, 229494, 30.08),
(2565.41, 'A069260', '휴켐스', 22700, 600, 2.71, 22700, 22750, 2099673, 78775, 19.0),
(2557.87, 'A014440', '영보화학', 5610, 260, 4.86, 5600, 5610, 7517673, 282846, 39.23),
(1866.36, 'A010040', '한국내화', 4975, 170, 3.54, 4970, 4975, 4198427, 213513, 18.22),
(660.02, 'A096760', 'JW홀딩스', 4530, 130, -2.79, 4525, 4530, 675953, 88939, 16.12),
(623.59, 'A004540', '깨끗한나라', 6860, 130, 1.93, 6860, 6870, 1698781, 234771, 9.03),
(591.06, 'A002140', '고려산업', 4760, 355, 8.06, 4755, 4760, 3461390, 500883, 14.78),
(589.99, 'A093050', 'LF', 19050, 1050, -5.22, 19050, 19100, 405753, 58806, 23.75),
(540.28, 'A089470', 'HDC현대EP', 6230, 170, 2.81, 6220, 6230, 425658, 66480, 13.57),
(504.16, 'A302440', '